In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_theme("paper", "white")
# plt.rc('text', usetex=True)
# plt.rc('font', family='sans-serif')

In [ ]:
data = pd.read_csv(snakemake.input[0], index_col=0)
cols = ["Time", "Memory"]
df = data.melt(id_vars=data.columns.drop(cols), value_vars=cols, var_name="kind")

In [ ]:
if snakemake.wildcards["kind"] == "overhead":
    labels = ["Overhead time (s)", "Overhead memory (MB)"]
else:
    labels = ["Time (s)", "Memory (MB)"]

g = sns.FacetGrid(data=df, row="kind", sharey=False, height=2.0, aspect=2)
g.map_dataframe(
    sns.lineplot,
    x="Number of Variables",
    y="value",
    hue="API",
    style="API",
    marker=".",
    legend="full",
    zorder=8,
)
for ax, label in zip(g.axes.ravel(), labels):
    ax.set_ylabel(label)
    ax.set_title("")
    ax.grid(axis="y", lw=0.2, color="grey", zorder=3, alpha=0.4)
g.fig.tight_layout()
g.add_legend()
g.fig.savefig(
    snakemake.output.time_memory, bbox_inches="tight", pad_inches=0.1, dpi=300
)

In [ ]:
if snakemake.wildcards["kind"] == "overhead":
    label = "Computational overhead [MBs]"
else:
    label = "Computational resource [MBs]"

df = data.assign(Resource=data["Time"] * data["Memory"])
cols = ["Resource"]
df = df.melt(id_vars=df.columns.drop(cols), value_vars=cols, var_name="kind")

fig, ax = plt.subplots(figsize=(6, 3))
sns.lineplot(
    data=df,
    x="Number of Variables",
    y="value",
    hue="API",
    style="API",
    marker=".",
    legend="full",
    zorder=8,
)
sns.despine()
ax.set_ylabel(label)
ax.set_title("")
plt.ticklabel_format(axis="both", style="sci", scilimits=(3, 3))
ax.grid(axis="y", lw=0.2, color="grey", zorder=3, alpha=0.4)
fig.tight_layout()
fig.savefig(snakemake.output.resource, bbox_inches="tight", pad_inches=0.1, dpi=300)